In [1]:
!pip install python-dotenv

In [3]:
!pip install -qU pypdf

In [4]:
!pip install langchain-community

In [5]:
!pip install langchain

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document

# Load the PDF file
loader = PyPDFLoader("PPIP_Kelas_3.pdf")

# Load all pages from the PDF
docs = loader.load()

# Convert each page into a Document object with metadata
documents = []
for i, page in enumerate(docs):
    document = Document(
        page_content=page.page_content,  # Extract the text content of the page
        metadata={"source": "PPIP_Kelas_3.pdf", "page_number": i + 1}  # Add metadata
    )
    documents.append(document)

# Optional: Process all the Document objects
for doc in documents:
    print(f"Page {doc.metadata['page_number']} content: {doc.page_content[:100]}...")

Page 1 content: untuk Siswa SD/MI Kelas III
BADAN PEMBINAAN IDEOLOGI PANCASILA
2022...
Page 2 content: ii
Penaian: Buku ini merupakan buku referensi (rujukan) yang dis usun dan ditelaah 
oleh berbagai pi...
Page 3 content: iii
SAMBUTAN
Salam Pancasila!
Buku Pendidikan dan Pembinaan Ideologi Pancasila ini 
merupakan buku r...
Page 4 content: iv
Merdeka Belajar yang menjadi arus utama dalam pendidik an 
Indonesia saat ini. Untuk itu, dilakuk...
Page 5 content: v
pendidikan formal jenjang PAUD, SD/MI, SMP/MTs, dan S MA/MA/
MAK ini, saya sampaikan terima kasih ...
Page 6 content: vi
KEMENTERIAN PENDIDIKAN, KEBUDAYAAN, RISET, DAN TEKNOLOGI
 REPUBLIK INDONESIA 
KATA PENGANTAR
Keme...
Page 7 content: vii
memuat materi untuk mengembangkan sikap, pengetahuan, dan 
keterampilan  peserta didik, pendidik...
Page 8 content: viii
PETUNJUK 
PENGGUNAAN BUKU
Buku ini pada dasarnya merupakan dorongan atau stimulu s 
pembelajara...
Page 9 content: ix
PENGANT AR
TELADAN BERP ANCASILA
BELAJAR BERP ANCASILA

In [7]:
%pip install -qU langchain-openai

In [8]:
# Split
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
splits = text_splitter.split_documents(docs)

print (f"Your {len(docs)} documents have been split into {len(splits)} chunks")

Your 144 documents have been split into 147 chunks


In [9]:
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_api_key = ""

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [21]:
!pip install chromadb
if 'vectordb' in globals(): # If you've already made your vectordb this will delete it so you start fresh
    vectordb.delete_collection()

embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
vector_store = Chroma.from_documents(documents=splits, embedding=embeddings)

In [13]:
#Prompting for Naive RAG, query come from user question only no added prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "Pengguna adalah siswa sekolah dasar yang sedang belajar tentang pendidikan dan pembinaan ideologi pancasila",
     "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: Pengguna adalah siswa sekolah dasar yang sedang belajar tentang pendidikan dan pembinaan ideologi pancasila 
Answer:


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

In [15]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [16]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

In [19]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [18]:
pip install -U langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.2 MB/s eta 0:00:00


In [22]:
result = graph.invoke({"question": "Siapa ando dan dila?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

Context: [Document(metadata={'creationdate': '2023-01-25T16:14:19+07:00', 'creator': 'PyPDF', 'moddate': '2023-01-25T16:14:19+07:00', 'page': 15, 'page_label': '16', 'producer': 'GPL Ghostscript 9.05', 'source': 'PPIP_Kelas_3.pdf', 'total_pages': 144}, page_content='Pendidikan dan Pembinaan Ideologi Pancasila untuk Siswa SD/MI Kelas III4\nAndo dan Dila gemar \nbelajar sejarah. Mereka \nselalu ingin tahu kejadian-\nkejadian di masa lalu. \nSaat ini mereka  sedang \nbelajar tentang sejarah \nlahirnya Pancasila. \nAndo dan Dila  ingin tahu \nbagaimana Pancasila lahir dan menjadi dasar negara.\n“Wah, ternyata yang pertama kali mengusulkan nama \nPancasila adalah Bapak Sukarno!” Ando mengatakan itu \nkepada Rania. \n“Bapak Sukarno, Presiden pertama Indonesia?”  \n tanya Dila.\n“Benar sekali. Tetapi, pada saat Sukarno mengusulkan \nPancasila, beliau belum menjadi presiden.” \n“Bagaimana ceritanya usul Sukarno bisa diterima?” \ntanya Dila, lagi.\n“Sebaiknya nanti kita tanyakan kepada Ibu Guru

In [ ]:
result = graph.invoke({"question": "What is BPUPK?"})

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')